# This Notebook was run on Paperspace Gradient 

**The GPU used to run this notebook is Nvidia Quadro RTX5000**

In [1]:
#CUDA version check. Here its 11.2
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
#Check GPU Type & Specs
!nvidia-smi

Tue Aug  9 13:37:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     Off  | 00000000:00:05.0 Off |                  Off |
| 33%   27C    P8    20W / 230W |      0MiB / 16125MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Tools & Models Installation

In [3]:
#Install spaCy for CUDA 11.2 & CUPY compatible version
!pip install -U spacy[cuda112,transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.0
    Uninstalling spacy-3.4.0:
      Successfully uninstalled spacy-3.4.0


In [4]:
#Download en_core_web_trf model
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [5]:
#Install PyTorch version compatible with CUDA 11.1 
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 GB 1.0 MB/s eta 0:00:00:00:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 17.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu116
    Uninstalling torch-1.12.0+cu116:
      Successfully uninstalled torch-1.12.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu116
    Uninstalling torchvision-0.13.0+cu116:
      Successfully uninstalled torchvision-0.13.0+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.0+cu116
    Uninstalling torchaudio-0.12.0+cu116:
      Successfully uninstalled torchaudio-0.12.0+cu116


In [6]:
import torch
torch.cuda.is_available()

True

---

# Libraries & Constants

In [7]:
import spacy
import spacy_transformers
import pandas as pd
from tqdm import tqdm
import json
import time

---

# \~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~ NER Training \~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~\~

## ============================ Experiment 3 ============================

## Transformers Model using "en_core_web_trf"

->->->   **The pipeline used here is: [transformer, ner], & its optimized for accuracy in the config file.**

* The folowing model is run on the following sample sizes: 
    * Training: 2000 samples
    * Validation: 700 samples
    * Testing: 235 samples

### Initialize Config File

In [8]:
#Initialize config file with default parameters of BERT
!python -m spacy init fill-config /notebooks/config/trf_base_config_accuracy.cfg /notebooks/config/trf_config_acc.cfg

✔ Auto-filled config with all values
✔ Saved config
/notebooks/config/trf_config_acc.cfg
You can now add your data and train your pipeline:
python -m spacy train trf_config_acc.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Debug Data

In [9]:
# Debug & validate the training and development data, get useful stats, 
# and find problems like invalid entity annotations,cyclic dependencies, low data labels and more.
!python -m spacy debug data /notebooks/config/trf_config_acc.cfg --paths.train /notebooks/data/processed/ner_for_training/spacy_train_mini.spacy --paths.dev /notebooks/data/processed/ner_for_training/spacy_dev_mini.spacy


============================ Data file validation ============================
Downloading: 100%|██████████████████████████████| 481/481 [00:00<00:00, 552kB/s]
Downloading: 100%|███████████████████████████| 878k/878k [00:00<00:00, 14.0MB/s]
Downloading: 100%|███████████████████████████| 446k/446k [00:00<00:00, 10.1MB/s]
Downloading: 100%|█████████████████████████| 1.29M/1.29M [00:00<00:00, 16.7MB/s]
Downloading: 100%|███████████████████████████| 478M/478M [00:21<00:00, 23.5MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initia

### Train NER Transformer Model

In [12]:
start = time.time()

!python -m spacy train -g 0 /notebooks/config/trf_config_acc.cfg --output /notebooks/models/trf_model_2 --paths.train /notebooks/data/processed/ner_for_training/spacy_train_mini.spacy --paths.dev /notebooks/data/processed/ner_for_training/spacy_dev_mini.spacy 

ℹ Saving to output directory: /notebooks/models/trf_model_2
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-09 13:45:20,739] [INFO] Set up nlp object from config
[2022-08-09 13:45:20,749] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-09 13:45:20,752] [INFO] Created vocabulary
[2022-08-09 13:45:20,753] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly ide

In [ ]:
elapsed_time = time.time() - start
print('Transformer Model 2 Training Time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Transformer Model 2 Training Time: 01:25:46


---

## Test the model

In [14]:
!python -m spacy evaluate /notebooks/models/trf_model_2/model-best /notebooks/data/processed/ner_for_training/spacy_test_mini.spacy -g 0 -o /notebooks/evaluation/eval_trf_2.json -dp /notebooks/figures/trf_model_2

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   76.61 
NER R   70.37 
NER F   73.35 
SPEED   12308 


=============================== NER (per type) ===============================

            P       R       F
LOC     71.91   71.91   71.91
DATE    83.59   85.26   84.42
ORG     72.05   59.28   65.05
FAC     75.96   71.82   73.83
MONEY   80.00   83.33   81.63
EVENT   71.43   41.67   52.63

/usr/local/lib/python3.9/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
✔ Generated 25 parses as HTML
/notebooks/figures/trf_model_2
✔ Saved results to /notebooks/evaluation/eval_trf_2.json
